In [4]:
!pip install dash-bootstrap-components

In [5]:
import os
print("Current directory:", os.getcwd())
print("Files in current directory:", os.listdir())


Current directory: /content
Files in current directory: ['.config', 'sample_data']


In [7]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

# تحميل البيانات
df = pd.read_csv("/clean_data_for_cafe_sales.csv")

# تحويل التاريخ وتوليد عمود الشهر
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])
df['Month'] = df['Transaction Date'].dt.to_period('M').astype(str)

# إنشاء التطبيق
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = "Cafe Sales Dashboard"

# تخطيط الواجهة
app.layout = dbc.Container([
    html.H1("Cafe Sales Dashboard", className="text-center my-4"),

    # مؤشرات الأداء
    dbc.Row([
        dbc.Col(html.Div(id="total-revenue", className="text-center bg-light p-3 rounded shadow-sm"), width=4),
        dbc.Col(html.Div(id="total-orders", className="text-center bg-light p-3 rounded shadow-sm"), width=4),
        dbc.Col(html.Div(id="total-items", className="text-center bg-light p-3 rounded shadow-sm"), width=4),
    ], className="mb-4"),

    # الفلاتر
    dbc.Row([
        dbc.Col([
            html.Label("Select Item:"),
            dcc.Dropdown(
                options=[{"label": i, "value": i} for i in df['Item'].unique()],
                id="item-filter",
                multi=True
            )
        ], width=4),
        dbc.Col([
            html.Label("Select Payment Method:"),
            dcc.Dropdown(
                options=[{"label": i, "value": i} for i in df['Payment Method'].unique()],
                id="payment-filter",
                multi=True
            )
        ], width=4),
        dbc.Col([
            html.Label("Select Date Range:"),
            dcc.DatePickerRange(
                id="date-filter",
                start_date=df['Transaction Date'].min(),
                end_date=df['Transaction Date'].max(),
                display_format="YYYY-MM-DD"
            )
        ], width=4),
    ], className="mb-4"),

    # الرسوم البيانية
    dbc.Row([
        dbc.Col(dcc.Graph(id="sales-by-item"), width=6),
        dbc.Col(dcc.Graph(id="sales-over-time"), width=6),
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id="payment-methods"), width=6),
        dbc.Col(dcc.Graph(id="location-distribution"), width=6),
    ]),
], fluid=True)

# التفاعل
@app.callback(
    [Output("total-revenue", "children"),
     Output("total-orders", "children"),
     Output("total-items", "children"),
     Output("sales-by-item", "figure"),
     Output("sales-over-time", "figure"),
     Output("payment-methods", "figure"),
     Output("location-distribution", "figure")],
    [Input("item-filter", "value"),
     Input("payment-filter", "value"),
     Input("date-filter", "start_date"),
     Input("date-filter", "end_date")]
)
def update_dashboard(selected_items, selected_payments, start_date, end_date):
    dff = df.copy()
    if selected_items:
        dff = dff[dff['Item'].isin(selected_items)]
    if selected_payments:
        dff = dff[dff['Payment Method'].isin(selected_payments)]
    dff = dff[(dff['Transaction Date'] >= start_date) & (dff['Transaction Date'] <= end_date)]

    # KPIs
    total_revenue = f"Total Revenue: ${dff['Total Spent'].sum():,.2f}"
    total_orders = f"Total Orders: {dff['Transaction ID'].nunique():,}"
    total_items = f"Total Items Sold: {int(dff['Quantity'].sum()):,}"

    # Graphs
    fig_item = px.bar(
        dff.groupby('Item')['Total Spent'].sum().reset_index(),
        x='Item', y='Total Spent',
        title="Sales by Item", color='Total Spent', color_continuous_scale='Blues'
    )
    fig_time = px.line(
        dff.groupby('Month')['Total Spent'].sum().reset_index(),
        x='Month', y='Total Spent',
        title="Sales Over Time"
    )
    fig_payment = px.pie(
        dff, names='Payment Method', values='Total Spent',
        title="Payment Method Distribution"
    )
    fig_location = px.pie(
        dff, names='Location', values='Total Spent',
        title="Location Distribution"
    )

    return total_revenue, total_orders, total_items, fig_item, fig_time, fig_payment, fig_location

# لتشغيل التطبيق
if __name__ == '__main__':
    app.run(debug=True)


<IPython.core.display.Javascript object>

In [13]:
!pip install pyngrok
from pyngrok import ngrok
import threading


In [15]:
def run_app():
    app.run_server(port=8050)

thread = threading.Thread(target=run_app)
thread.start()

# إنشاء نفق ngrok
public_url = ngrok.connect(8050)
print("Dashboard URL:", public_url)

Exception in thread Thread-314 (run_app):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-15-2861e07b0252>", line 2, in run_app
  File "/usr/local/lib/python3.11/dist-packages/dash/_obsolete.py", line 22, in __getattr__
    raise err.exc(err.message)
dash.exceptions.ObsoleteAttributeException: app.run_server has been replaced by app.run


Dashboard URL: NgrokTunnel: "https://54b7-35-230-30-84.ngrok-free.app" -> "http://localhost:8050"
